In [1]:
import os
import numpy as np

path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
#path = '.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset'

files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if ".avi" in file:
            files.append(os.path.join(r, file))
        
for f in files:
    print(f)
np.array(files).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\slowBlinkWithNodding.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\yawning.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\slowBlinkWithNodding.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\yawning.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\nonsleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\sleepyCombination.avi
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\slow

(356,)

In [2]:
Statelist=['nonsleepyCombination','sleepyCombination','slowBlinkWithNodding','yawning']
for i in range(4):
    print(files[i][55:58])
    print(files[i][67:-4])
    print(files[i][54:-4])
    print('----')

001
nonsleepyCombination
\001\glasses\nonsleepyCombination
----
001
sleepyCombination
\001\glasses\sleepyCombination
----
001
slowBlinkWithNodding
\001\glasses\slowBlinkWithNodding
----
001
yawning
\001\glasses\yawning
----


## MTCNN

if face not detected in 60 frames, use the last detected 

In [3]:
from mtcnn.mtcnn import MTCNN
# import the necessary packages
from imutils import face_utils
import dlib
import cv2
import numpy as np
import os
from operator import itemgetter
import time

eyes_width = 60
eyes_height = 33
mouth_width = 50
mouth_height = 33
face_width = 66
face_height = 85
num_classes = 2
channel = 1

eyes_width2 = 45
eyes_height2 = 25
mouth_width2 = 35
mouth_height2 = 25
face_width2 = 50
face_height2 = 64



def CreateFolder(name):
    try:
        # creating a folder named data 
        if not os.path.exists(name):
            os.makedirs(name)
        # if not created then raise error 
    except OSError: 
        print ('Error: Creating directory of data')            
#==============================================================

detector = MTCNN()

for filename in files:
    
    #==========================
    f = open('droplist.txt','a')
    f.write(filename+"\n")
    f.close()
    #==========================
    # img = cv2.imread("me.jpg")
    
    #filename = '.\\Dataset\\Training_Evaluation_Dataset\\Training Dataset\\001\\glasses\\yawning.avi'

    print(filename)
    cap = cv2.VideoCapture(filename)
    #================================================================ create new folder

    DefaultPath = '.\\Dataset\\Training_Evaluation_Dataset\\Training Image\\'+filename[54:-4]+"\\"
    CreateFolder(DefaultPath+"face")
    CreateFolder(DefaultPath+"eyes")
    CreateFolder(DefaultPath+"mouth")
    #================================================================
    Face_ImageDataList = []

    currentframe = 0
    missing_frame = 0
    DropList = []

    eyes_w = eyes_width
    eyes_h = eyes_height
    face_w = face_width
    face_h = face_height
    mouth_w = mouth_width
    mouth_h = mouth_height
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(length)
    x,y,w,h = 0,0,0,0
    
    while(cap.isOpened()):
        
        currentframe += 1
        # load the input image and convert it to grayscale
        ret, image = cap.read()
        if (not ret):
          break
        
        start_time = time.time()
        

        
        #3 frame sample once
        if(currentframe%3!=0):
            continue
        
        #if(currentframe<2200):
        #    continue
        

        height,width = image.shape[:2]
        #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #image = cv2.resize(image,(width//2,height//2),interpolation=cv2.INTER_CUBIC)
        #print(image.shape[:2])
        faces = detector.detect_faces(image)
        #print(np.array(faces).shape[0])
        
        if(np.array(faces).shape[0] > 0):
            face = faces[0]
            x,y,w,h = face['box']
            
            face_image = image[max(0,y):max(0,y+h),max(0,x):max(0,x+w)]
            #print(face_image.shape)
            #print(face_image.shape)
            #print(image[x:x+w,y:y+h])
            #cv2.rectangle(image,(x,y),(x+w,y+h),(255,0,0),2)
            face_image1 = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame_Middle" + str(currentframe) + '.jpg',face_image1)
            
            face_image2 = cv2.resize(face_image,(face_width2,face_height2),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame_Small" + str(currentframe) + '.jpg',face_image2) 


            eyes_image = image[max(0,face['keypoints']['left_eye'][1]-int(h*0.2)):max(0,face['keypoints']['right_eye'][1]+int(h*0.2))
                                ,max(0,face['keypoints']['left_eye'][0]-int(w*0.2)):max(0,face['keypoints']['right_eye'][0]+int(w*0.2))]
            
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image1 = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame_Middle" + str(currentframe) + '.jpg',eyes_image1) 
            
            eyes_image2 = cv2.resize(eyes_image,(eyes_width2,eyes_height2),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame_Small" + str(currentframe) + '.jpg',eyes_image2) 


            mouth_image = image[max(0,face['keypoints']['mouth_left'][1]-int(h*0.2)):max(0,face['keypoints']['mouth_right'][1]+int(h*0.2))
                               ,max(0,face['keypoints']['mouth_left'][0]-int(w*0.2)):max(0,face['keypoints']['mouth_right'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image1 = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame_Middle" + str(currentframe) + '.jpg',mouth_image1) 
            
            mouth_image2 = cv2.resize(mouth_image,(mouth_width2,mouth_height2),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame_Small" + str(currentframe) + '.jpg',mouth_image2) 



            #cv2.rectangle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)


            #cv2.circle(image, (face['keypoints']['left_eye'][0], face['keypoints']['left_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['right_eye'][0], face['keypoints']['right_eye'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['nose'][0], face['keypoints']['nose'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_left'][0], face['keypoints']['mouth_left'][1]), 2, (0, 255, 0), -1)
            #cv2.circle(image, (face['keypoints']['mouth_right'][0], face['keypoints']['mouth_right'][1]), 2, (0, 255, 0), -1)
            if(currentframe%100==0):        
                name = './data/frame' + str(currentframe) + '.jpg'
                print ('Creating...' + name) 
        elif (missing_frame<60 and (not((x == 0) or (y == 0) or (w == 0) or (h == 0) ))):
            print ('missing using last frame') 
            missing_frame = missing_frame + 1        

            face_image = image[max(0,y):max(0,y+h),max(0,x):max(0,x+w)]
            
            face_image1 = cv2.resize(face_image,(face_w,face_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame_Middle" + str(currentframe) + '.jpg',face_image1)
            
            face_image2 = cv2.resize(face_image,(face_width2,face_height2),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "face\\frame_Small" + str(currentframe) + '.jpg',face_image2) 


            eyes_image = image[max(0,face['keypoints']['left_eye'][1]-int(h*0.2)):max(0,face['keypoints']['right_eye'][1]+int(h*0.2))
                                ,max(0,face['keypoints']['left_eye'][0]-int(w*0.2)):max(0,face['keypoints']['right_eye'][0]+int(w*0.2))]
            
            #cv2.rectangle(image, (face['keypoints']['left_eye'][0]-int(w*0.2), face['keypoints']['left_eye'][1]-int(h*0.2)),
            #              (face['keypoints']['right_eye'][0]+int(w*0.2), face['keypoints']['right_eye'][1]+int(h*0.2)),(0, 255, 0),2)
            eyes_image1 = cv2.resize(eyes_image,(eyes_w,eyes_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame_Middle" + str(currentframe) + '.jpg',eyes_image1) 
            
            eyes_image2 = cv2.resize(eyes_image,(eyes_width2,eyes_height2),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "eyes\\frame_Small" + str(currentframe) + '.jpg',eyes_image2) 


            mouth_image = image[max(0,face['keypoints']['mouth_left'][1]-int(h*0.2)):max(0,face['keypoints']['mouth_right'][1]+int(h*0.2))
                               ,max(0,face['keypoints']['mouth_left'][0]-int(w*0.2)):max(0,face['keypoints']['mouth_right'][0]+int(w*0.2))]
            #cv2.rectangle(image, (face['keypoints']['mouth_left'][0]-int(w*0.2), face['keypoints']['mouth_left'][1]-int(h*0.2)),
            #              (face['keypoints']['mouth_right'][0]+int(w*0.2), face['keypoints']['mouth_right'][1]+int(h*0.2)),(0, 255, 0),2)
            mouth_image1 = cv2.resize(mouth_image,(mouth_w,mouth_h),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame_Middle" + str(currentframe) + '.jpg',mouth_image1) 
            
            mouth_image2 = cv2.resize(mouth_image,(mouth_width2,mouth_height2),interpolation=cv2.INTER_AREA)
            cv2.imwrite(DefaultPath + "mouth\\frame_Small" + str(currentframe) + '.jpg',mouth_image2) 
            
            #if(currentframe%100==0):
            name = './data/frame' + str(currentframe) + '.jpg'
            print ('missing using last frame:  '+ 'Creating...' + name) 

        else:
            name = './data/frame' + str(currentframe) + '.jpg'
            f = open('droplist.txt','a')
            f.write(name+"\n")
            f.close()
            DropList.append(currentframe)
            print("drop")



        '''
        cv2.imshow('img',image)

        #print("FPS: ", 1.0 / (time.time() - start_time))

        k = cv2.waitKey(5) & 0xFF
        if k == 27:
            break
        '''
    
    print("total_frame:" +str(length)+" missing frame"+ str(np.array(DropList).shape[0])+" total frame2:"+ str(currentframe))
    print(DropList)
    cap.release()
    cv2.destroyAllWindows()

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\nonsleepyCombination.avi
3330
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
Creating..../data/frame3000.jpg
Creating..../data/frame3300.jpg
total_frame:3330 missing frame0 total frame2:3331
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sleepyCombination.avi
2748
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
total_frame:2748 missing frame0 total frame2:2749
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\sl

Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
total_frame:2711 missing frame0 total frame2:2712
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\noglasses\slowBlinkWithNodding.avi
1852
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
total_frame:1852 missing frame0 total frame2:1853
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\noglasses\yawning.avi
1872
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
Creating..../data/frame900.jpg
Creating..../data/frame1200.jpg
Creating..../data/frame1500.jpg
Creating..../data/frame1800.jpg
total_frame:1872 missing frame0 total frame2:1873
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\sunglasses\nonsleepyCombination.avi
2803
Creating..../data/

missing using last frame
missing using last frame:  Creating..../data/frame291.jpg
missing using last frame
missing using last frame:  Creating..../data/frame294.jpg
missing using last frame
missing using last frame:  Creating..../data/frame297.jpg
missing using last frame
missing using last frame:  Creating..../data/frame300.jpg
missing using last frame
missing using last frame:  Creating..../data/frame303.jpg
missing using last frame
missing using last frame:  Creating..../data/frame306.jpg
missing using last frame
missing using last frame:  Creating..../data/frame309.jpg
missing using last frame
missing using last frame:  Creating..../data/frame312.jpg
missing using last frame
missing using last frame:  Creating..../data/frame315.jpg
missing using last frame
missing using last frame:  Creating..../data/frame318.jpg
missing using last frame
missing using last frame:  Creating..../data/frame321.jpg
missing using last frame
missing using last frame:  Creating..../data/frame324.jpg
miss

missing using last frame
missing using last frame:  Creating..../data/frame855.jpg
missing using last frame
missing using last frame:  Creating..../data/frame858.jpg
missing using last frame
missing using last frame:  Creating..../data/frame861.jpg
missing using last frame
missing using last frame:  Creating..../data/frame864.jpg
missing using last frame
missing using last frame:  Creating..../data/frame867.jpg
missing using last frame
missing using last frame:  Creating..../data/frame870.jpg
Creating..../data/frame900.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1032.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1035.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1038.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1041.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1044.jpg
missing using last frame
missing using last frame: 

...

Creating..../data/frame2100.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2298.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2304.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2307.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2310.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2313.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2316.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2319.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2322.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2325.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2328.jpg
missing using last frame
missing using last frame:  Creating..../data/frame2331.jpg
Creating..../data/frame2400.jpg
total_frame:

missing using last frame
missing using last frame:  Creating..../data/frame390.jpg
missing using last frame
missing using last frame:  Creating..../data/frame393.jpg
missing using last frame
missing using last frame:  Creating..../data/frame396.jpg
missing using last frame
missing using last frame:  Creating..../data/frame399.jpg
missing using last frame
missing using last frame:  Creating..../data/frame402.jpg
missing using last frame
missing using last frame:  Creating..../data/frame405.jpg
missing using last frame
missing using last frame:  Creating..../data/frame408.jpg
missing using last frame
missing using last frame:  Creating..../data/frame411.jpg
missing using last frame
missing using last frame:  Creating..../data/frame414.jpg
Creating..../data/frame600.jpg
missing using last frame
missing using last frame:  Creating..../data/frame774.jpg
missing using last frame
missing using last frame:  Creating..../data/frame777.jpg
missing using last frame
missing using last frame:  Crea

missing using last frame:  Creating..../data/frame1404.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1407.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1410.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1413.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1416.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1419.jpg
Creating..../data/frame1500.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1527.jpg
drop
drop
total_frame:1558 missing frame2 total frame2:1559
[1542, 1545]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\036\night_noglasses\sleepyCombination.avi
1508
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
missing using last frame
missing using last frame:  Creating..../data/frame708.jpg
missing using last frame
missing using last frame:  Creating..../data/frame717.jpg
mi

Creating..../data/frame900.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1059.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1062.jpg
Creating..../data/frame1200.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1230.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1233.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1239.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1242.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1245.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1248.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1488.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1494.jpg
missing using last frame
missing using last frame:  Creating..../data/frame1497.jpg
Creating..../

Creating..../data/frame1800.jpg
Creating..../data/frame2100.jpg
Creating..../data/frame2400.jpg
Creating..../data/frame2700.jpg
total_frame:2739 missing frame0 total frame2:2740
[]
.\Dataset\Training_Evaluation_Dataset\Training Dataset\036\sunglasses\slowBlinkWithNodding.avi
2259
Creating..../data/frame300.jpg
Creating..../data/frame600.jpg
missing using last frame
missing using last frame:  Creating..../data/frame687.jpg
missing using last frame
missing using last frame:  Creating..../data/frame699.jpg
missing using last frame
missing using last frame:  Creating..../data/frame702.jpg
missing using last frame
missing using last frame:  Creating..../data/frame708.jpg
missing using last frame
missing using last frame:  Creating..../data/frame711.jpg
missing using last frame
missing using last frame:  Creating..../data/frame714.jpg
missing using last frame
missing using last frame:  Creating..../data/frame717.jpg
missing using last frame
missing using last frame:  Creating..../data/frame7

### TODO
1. 所有file 存檔案
2. training 進CNN model
3. 存沒有東西的frame

### define CNN [model](https://medium.com/@mannasiladittya/building-inception-resnet-v2-in-keras-from-scratch-a3546c4d93f0)


In [1]:
%run ./Model.ipynb 

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:

eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [3]:
img_input = Input(shape=(face_height,face_width,1))
#img_input = Input(shape=(face_height,face_width,1))
#img_input = Input(shape=(face_height,face_width,1))

x = conv2d(img_input,32,3,2,'valid',True,name='conv1')
x = conv2d(x,32,3,1,'valid',True,name='conv2')
x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x_12 = conv2d(x,96,3,1,'valid',True,name='stem_br_12')

x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])

In [4]:
#Inception-ResNet-A modules
x = incresA(x,0.15,name='incresA_1')
x = incresA(x,0.15,name='incresA_2')
x = incresA(x,0.15,name='incresA_3')
#x = incresA(x,0.15,name='incresA_4')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

#Inception-ResNet-B modules
x = incresB(x,0.1,name='incresB_1')
x = incresB(x,0.1,name='incresB_2')
x = incresB(x,0.1,name='incresB_3')
x = incresB(x,0.1,name='incresB_4')
x = incresB(x,0.1,name='incresB_5')
#x = incresB(x,0.1,name='incresB_6')
#x = incresB(x,0.1,name='incresB_7')

#17 × 17 to 8 × 8 reduction module.
x_red_21 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

x_red_22 = conv2d(x,256,1,1,'same',True,name='x_red2_c11')
x_red_22 = conv2d(x_red_22,384,3,2,'valid',True,name='x_red2_c12')

x_red_23 = conv2d(x,256,1,1,'same',True,name='x_red2_c21')
x_red_23 = conv2d(x_red_23,256,3,2,'valid',True,name='x_red2_c22')

x_red_24 = conv2d(x,256,1,1,'same',True,name='x_red2_c31')
x_red_24 = conv2d(x_red_24,256,3,1,'same',True,name='x_red2_c32')
x_red_24 = conv2d(x_red_24,256,3,2,'valid',True,name='x_red2_c33')

x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])

#Inception-ResNet-C modules
x = incresC(x,0.2,name='incresC_1')
x = incresC(x,0.2,name='incresC_2')
#x = incresC(x,0.2,name='incresC_3')

#TOP
x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dropout(0.6)(x)
#x = Dense(num_classes, activation='softmax')(x)
flat3 = Flatten()(x)

In [5]:


'''
# first input model: eyes
eyes1 = Input(shape=(eyes_height,eyes_width,1))
conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(eyes1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
#pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
#flat1 = Flatten()(pool12)
pool12 = GlobalAveragePooling2D(data_format='channels_last')(conv12)
flat1 = Flatten()(pool12)


# second input model: mouth
mouth1 = Input(shape=(mouth_height,mouth_width,1))
conv21 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(16, kernel_size=(3, 3), activation='relu')(pool21)
#pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
#flat2 = Flatten()(pool22)
pool22 = GlobalAveragePooling2D(data_format='channels_last')(conv22)
flat2 = Flatten()(pool22)
'''
#merge = Concatenate(axis=1, name='concat_2')([flat1, flat2,flat3])
output = Dense(num_classes, activation='softmax')(flat3)

In [6]:
model = Model(inputs=[ img_input], outputs=output, name='inception_resnet_v2')
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 31, 24, 32)   288         input_1[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 31, 24, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 31, 24, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
'''
from keras.utils.vis_utils import model_to_dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

SVG(model_to_dot(model).create(prog='dot', format='svg'))
plot_model(model, to_file='model_plot.png', 
                  show_shapes=True, show_layer_names=True)
'''

"\nfrom keras.utils.vis_utils import model_to_dot\nfrom tensorflow.keras.utils import plot_model\nfrom IPython.display import SVG\n\nSVG(model_to_dot(model).create(prog='dot', format='svg'))\nplot_model(model, to_file='model_plot.png', \n                  show_shapes=True, show_layer_names=True)\n"

### Model2

In [9]:

# first input model: eyes
eyes1 = Input(shape=(eyes_height,eyes_width,1))
conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(eyes1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
#pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
#flat1 = Flatten()(pool12)
pool12 = GlobalAveragePooling2D(data_format='channels_last')(conv12)
flat1 = Flatten()(pool12)


# second input model: mouth
mouth1 = Input(shape=(mouth_height,mouth_width,1))
conv21 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool21)
#pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
#flat2 = Flatten()(pool22)
pool22 = GlobalAveragePooling2D(data_format='channels_last')(conv22)
flat2 = Flatten()(pool22)

# input model: face
face1 = Input(shape=(face_height,face_width,1))
conv31 = Conv2D(32, kernel_size=(3, 3), activation='relu')(face1)
pool31 = MaxPooling2D(pool_size=(2, 2))(conv31)
x = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool31)
#pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
#flat2 = Flatten()(pool22)
'''
x = incresA(face1,0.15,name='incresA_1')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

x = incresB(x,0.1,name='incresB_1')
'''

x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dropout(0.6)(x)
flat3 = Flatten()(x)

merge = Concatenate(axis=1, name='concat_2')([flat1, flat2,flat3])
output = Dense(num_classes, activation='softmax')(merge)

In [10]:
model2 = Model(inputs=[eyes1, mouth1, face1], outputs=output, name='inception_resnet_v2')
model2.summary()
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 25, 45, 1)    0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 25, 35, 1)    0                                            
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 62, 48, 32)   320         input_6[0][0]                    
__________________________________________________________________________________________________
conv2d_4 (

### model3

In [11]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import applications
from tensorflow.keras.layers import Input


eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

img_width, img_height = face_width, face_height

face1 = Input(shape=(face_height,face_width,1))

vgg_model = applications.VGG16(weights='imagenet', include_top=False,input_shape = (img_width, face_height, 3))
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 50, 64, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 50, 64, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 50, 64, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 25, 32, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 25, 32, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 25, 32, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 12, 16, 128)       0         
__________

In [12]:
#在VGG16的block2_pool output再加layer，VGG16的3 4 5就不要了
layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])
layer_dict

{'input_8': <tensorflow.python.keras.engine.input_layer.InputLayer at 0x1ec491d09e8>,
 'block1_conv1': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec49205940>,
 'block1_conv2': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec492055f8>,
 'block1_pool': <tensorflow.python.keras.layers.pooling.MaxPooling2D at 0x1ec49205a58>,
 'block2_conv1': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec49205fd0>,
 'block2_conv2': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec49226710>,
 'block2_pool': <tensorflow.python.keras.layers.pooling.MaxPooling2D at 0x1ec4923de80>,
 'block3_conv1': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec4923de48>,
 'block3_conv2': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec4927f9e8>,
 'block3_conv3': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec492c7940>,
 'block3_pool': <tensorflow.python.keras.layers.pooling.MaxPooling2D at 0x1ec492dbb00>,
 'block4_conv1': <tensorflo

In [13]:
#Functional API

# first input model: eyes
eyes1 = Input(shape=(eyes_height,eyes_width,1))
conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(eyes1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
#pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
#flat1 = Flatten()(pool12)
pool12 = GlobalAveragePooling2D(data_format='channels_last')(conv12)
flat1 = Flatten()(pool12)


# second input model: mouth
mouth1 = Input(shape=(mouth_height,mouth_width,1))
conv21 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool21 = MaxPooling2D(pool_size=(2, 2))(conv21)
conv22 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool21)
#pool22 = MaxPooling2D(pool_size=(2, 2))(conv22)
#flat2 = Flatten()(pool22)
pool22 = GlobalAveragePooling2D(data_format='channels_last')(conv22)
flat2 = Flatten()(pool22)

#===================================================



Conv2_out = layer_dict['block2_pool'].output

Conv3_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv2_out)
Conv3_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv3_out)
Conv3_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv3_out)
Conv3_Poolout = MaxPooling2D(pool_size=(2, 2))(Conv3_out)

Conv4_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv3_Poolout)
Conv4_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv4_out)
Conv4_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv4_out)
Conv4_Poolout = MaxPooling2D(pool_size=(2, 2))(Conv4_out)

Conv5_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv4_Poolout)
Conv5_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv5_out)
Conv5_out = Conv2D(filters=256, kernel_size=(1, 1), activation='relu', padding='same',kernel_initializer='random_uniform')(Conv5_out)
Conv5_Poolout = MaxPooling2D(pool_size=(2, 2))(Conv5_out)


FC6_1 = MaxPooling2D(pool_size=(12, 16))(Conv3_out)
FC6_2 = MaxPooling2D(pool_size=(6, 8))(Conv4_out)
FC6_3 = MaxPooling2D(pool_size=(3, 4))(Conv5_out)

FC6_in=Concatenate(axis=1, name='concat_2')([FC6_1,FC6_2,FC6_3])

out = Flatten()(FC6_in)

out = Dense(256, activation='relu',kernel_initializer='random_uniform')(out)
out = Dense(256, activation='relu',kernel_initializer='random_uniform')(out)
out = Dense(2, activation='softmax',kernel_initializer='random_uniform')(out)


In [14]:
from keras.models import Model
from keras import optimizers
custom_model = Model(input=vgg_model.input, output=out)

# Make sure that the pre-trained bottom layers are not trainable
for layer in custom_model.layers[:7]:
    layer.trainable = False
    print(layer)


sgd = optimizers.SGD(lr=0.000005, decay=0.0005, momentum=0.9, nesterov=True)
# Do not forget to compile it
custom_model.compile(loss='categorical_crossentropy',
                     optimizer='adam', 
                     metrics=['accuracy'])

custom_model.summary()

Using TensorFlow backend.
C:\Users\T-Rex\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  This is separate from the ipykernel package so we can avoid doing imports until


AttributeError: 'Tensor' object has no attribute '_keras_shape'

### model4

In [10]:
from tensorflow.keras.applications.densenet import DenseNet169


img_width, img_height = face_width, face_height

face1 = Input(shape=(face_height,face_width,1))

DenseNet169Model = applications.DenseNet169(weights='imagenet', include_top=False,input_shape = (face_height,img_width, 3))

NameError: name 'applications' is not defined

In [16]:
DenseNet169Model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 64, 50, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 56, 3)    0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 25, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 25, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [17]:
layer_dict = dict([(layer.name, layer) for layer in DenseNet169Model.layers])
layer_dict

{'input_12': <tensorflow.python.keras.engine.input_layer.InputLayer at 0x1ec49936c18>,
 'zero_padding2d': <tensorflow.python.keras.layers.convolutional.ZeroPadding2D at 0x1ec49936d30>,
 'conv1/conv': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec49a2d3c8>,
 'conv1/bn': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x1ec49a2d358>,
 'conv1/relu': <tensorflow.python.keras.layers.core.Activation at 0x1ec4992ceb8>,
 'zero_padding2d_1': <tensorflow.python.keras.layers.convolutional.ZeroPadding2D at 0x1ec499360b8>,
 'pool1': <tensorflow.python.keras.layers.pooling.MaxPooling2D at 0x1ec49a2d748>,
 'conv2_block1_0_bn': <tensorflow.python.keras.layers.normalization.BatchNormalization at 0x1ec49a9ed30>,
 'conv2_block1_0_relu': <tensorflow.python.keras.layers.core.Activation at 0x1ec49a2df28>,
 'conv2_block1_1_conv': <tensorflow.python.keras.layers.convolutional.Conv2D at 0x1ec49aedac8>,
 'conv2_block1_1_bn': <tensorflow.python.keras.layers.normalization.BatchN

In [31]:
from tensorflow.keras.models import Model


Conv2_out = layer_dict['conv5_block32_concat'].output
out = Flatten()(Conv2_out)
out = Dense(1024, activation='relu',kernel_initializer='random_uniform')(out)
out = Dense(512, activation='relu',kernel_initializer='random_uniform')(out)
out = Dense(2, activation='softmax',kernel_initializer='random_uniform')(out)


custom_model = Model(inputs = DenseNet169Model.input, outputs=out, name='inception_resnet_v2')

In [32]:
custom_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 64, 50, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d (ZeroPadding2D)  (None, 70, 56, 3)    0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 32, 25, 64)   9408        zero_padding2d[0][0]             
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 32, 25, 64)   256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [33]:
custom_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0005), metrics=['accuracy'])

### Load one file and label

In [1]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

AllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "head.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_head.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_sleepyCombinatio

(356,)

In [2]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [3]:
num_samples = 235578
batch_size = 200

In [4]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping




def LoadImageandLabel(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:

            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1

            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\face"


            #=====find all files in path=======    
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" in file:
                        temp = os.path.join(r, file)
                        
                        Facefiles.append(temp)
                        '''
                        Eyefiles.append(temp.replace("face", "eyes"))
                        Mouthfiles.append(temp.replace("face", "mouth"))
                        '''

            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            '''
            Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            '''
            for FileIndex in range(len(Facefiles)):
                #======================load label

                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 

                #print(framenumber)

                #if(judgeflag == 0 and TempLabelList[framenumber-1] == 1):
                #    judgeflag = 1
                    #print(framenumber-1)

                #Labelarray = np.array(LabelList)
                #print(TempLabelList[framenumber-1])
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=3)
                LabelList.append(Labelarray)
                
                


                    #Labelarray = np.array(LabelList)
                    #Labelarray = to_categorical(Labelarray, num_classes=2)

                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                #ImgRGB = cv2.merge((ImgGray,ImgGray,ImgGray))# this line make gray image to RGB
                #TEMP_ImageDataList.append(ImgRGB)  
                
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                #ImgGray = cv2.merge((ImgGray,ImgGray,ImgGray))
                
                #print(ImgGray.shape)
                Face_ImageDataList.append(ImgGray)
                #print(np.array(Face_ImageDataList).shape)
                '''
                ImgGray = cv2.imread(Mouthfiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Mouth_ImageDataList.append(ImgGray)


                ImgGray = cv2.imread(Eyefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Eyes_ImageDataList.append(ImgGray)
                '''
                 #====================================
               
                count = count +1
                if count>=batch_size:
                        #yield(np.array(Face_ImageDataList),np.array(LabelList))
                        yield ([np.array(Face_ImageDataList)],np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        LabelList = []
               
            '''
            print(np.array(Eyes_ImageDataList).shape)
            print(np.array(Face_ImageDataList).shape)
            print(np.array(Mouth_ImageDataList).shape)
            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            LabelList = []
            '''
            #return Face_ImageDataList , Mouth_ImageDataList , Eyes_ImageDataList
            



#for epoch in range(50):
   # index = 0
#for Video in AllImageFolder:

    #if(index == 0 ):
    #    index = index + 1
    #    continue
    #print("---")
    #=========================load file

   # faceImage = []
   # mouthImage = []
   # eyesImage = []


    #print("label: "+labelfile)
    #==================================
    #LoadImageandLabel(50,AllImageFolder,AllLabelFile)
    #faceImage,mouthImage,eyesImage = LoadImageandLabel(50,,AllImageFolder,AllLabelFile)

    #print(eyesImage)

    #print(np.array(eyesImage).shape)
    #print(np.array(faceImage).shape)
    #print(np.array(mouthImage).shape)

    #print("---")
    #print("---")
    #print("---")
    #print(faceImage[1])
    
    
#earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

model.fit_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile),
                steps_per_epoch=ceil(num_samples / batch_size),
                epochs=100,
                #callbacks=[earlyStopping],
                workers = 5
                )
    
#loss ,accuracy= model2.evaluate_generator(LoadImageandLabel(50,AllImageFolder,AllLabelFile), steps=5)
    
    #model_result = model2.fit([eyesImage,mouthImage,faceImage], Labelarray,batch_size=20,epochs=epoch+1, initial_epoch=epoch, shuffle=True,verbose = 1)



C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


NameError: name 'model' is not defined

In [12]:

model_json = model.to_json()
open('ModelFace.json', 'w').write(model_json)
model.save_weights('ModelFace.h5')



1.dataGenerator <br/>
2.Image 減小

In [48]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='auto')

model2.fit_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile),
                steps_per_epoch=ceil(num_samples / batch_size),
                epochs=100,
                callbacks=[earlyStopping],
                workers = 5
                )
    
loss ,accuracy= model2.evaluate_generator(LoadImageandLabel(50,AllImageFolder,AllLabelFile), steps=5)

NameError: name 'model2' is not defined

### Eyes

In [1]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Testing Image'

AllImageFolderTesting = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("mix" in directory):
            #print(directory)
            AllImageFolderTesting.append(os.path.join(r, directory))    
            
AllImageFolderTesting = AllImageFolderTesting[6:15]
for d in AllImageFolderTesting:
    print(d)
    
#llImageFolder = AllImageFolder[2:]
np.array(AllImageFolderTesting).shape

.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_sunglasses_mix


(9,)

In [2]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset'

AllLabelFileTesting = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "eye.txt" in file :
            AllLabelFileTesting.append(os.path.join(r, file))

AllLabelFileTesting = AllLabelFileTesting[6:15]           
for f in AllLabelFileTesting:
    print(f)
    
#AllLabelFile = AllLabelFile [2:]
np.array(AllLabelFileTesting).shape

.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_nightglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_night_noglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_noglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_sunglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_glasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_nightglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_night_noglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_noglasses_mixing_eye.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_sunglasses_mixing_eye.txt


(9,)

In [3]:


eyes_width = 90
eyes_height = 50
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [4]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping




def LoadImageandLabelTesting(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:
            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1

            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\eyes"
            

            #=====find all files in path=======    
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if "Bigframe" in file:
                        temp = os.path.join(r, file)
                        Facefiles.append(temp)
                        
                        #Eyefiles.append(temp.replace("face", "eyes"))
                        #Mouthfiles.append(temp.replace("face", "mouth"))

            
            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            #Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            #Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            #Mouthfiles=Mouthfiles[:-2]
            #Eyefiles=Eyefiles[:-2]
            Facefiles=Facefiles[:-2]
            '''
            print(len(Facefiles))
            print(len(TempLabelList))
            '''
            for FileIndex in range(len(Facefiles)):
                #======================load label

                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 
                
                #print(framenumber)

                #if(judgeflag == 0 and TempLabelList[framenumber-1] == 1):
                #    judgeflag = 1
                    #print(framenumber-1)

                #Labelarray = np.array(LabelList)
                #print(len(TempLabelList))
                #print(framenumber-1)
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=num_classes)
                LabelList.append(Labelarray)
                #print(LabelList)

                    #Labelarray = np.array(LabelList)
                    #Labelarray = to_categorical(Labelarray, num_classes=2)

                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                #ImgRGB = cv2.merge((ImgGray,ImgGray,ImgGray))# this line make gray image to RGB
                #TEMP_ImageDataList.append(ImgRGB)  
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                #print(ImgGray.shape)
                Face_ImageDataList.append(ImgGray)
                #print(np.array(Face_ImageDataList).shape)
                '''
                ImgGray = cv2.imread(Mouthfiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Mouth_ImageDataList.append(ImgGray)


                ImgGray = cv2.imread(Eyefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Eyes_ImageDataList.append(ImgGray)
                '''
                 #====================================
               
                count = count +1
                if count>=batch_size:

                        yield ([np.array(Face_ImageDataList)],np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        LabelList = []
                        
#      

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
%run ./Model.ipynb 

In [6]:
eyes_width = 90
eyes_height = 50
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [7]:
eyes1 = Input(shape=(eyes_height,eyes_width,1))

x = conv2d(eyes1,32,3,2,'valid',True,name='conv1')
x = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x = conv2d(eyes1,32,3,2,'valid',True,name='conv1')
x = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x = conv2d(eyes1,32,3,2,'valid',True,name='conv1')
x = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)

#x = conv2d(x,32,3,1,'valid',True,name='conv2')
#x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
#x_12 = conv2d(x,96,3,1,'valid',True,name='stem_br_12')

#x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

#x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
#x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
#x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
#x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

#x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
#x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

#x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

#x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
#x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
#x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])

#x = incresA(x,0.15,name = 'incresA_1')
#x = incresA(x,0.15,name = 'incresA_2')
#x = incresA(x,0.15,name = 'incresA_3')

'''conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
conv13 = Conv2D(128, kernel_size=(3, 3), activation='relu',padding = 'same')(pool12)
pool13 = MaxPooling2D(pool_size=(2, 2))(conv13)
#flat1 = Flatten()(pool12)
#pool14 = GlobalAveragePooling2D(data_format='channels_last')(conv14)
flat1 = Flatten()(pool13)
output = Dense(num_classes, activation='softmax')(flat1)

'''
flat1 = Flatten()(x)

output = Dense(num_classes, activation='softmax')(flat1)




EyeModel = Model(inputs=[eyes1], outputs=output, name='inception_resnet_v2')
EyeModel.summary()
EyeModel.compile(loss='binary_crossentropy', optimizer=Adam(lr = 0.001), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 50, 90, 1)         0         
_________________________________________________________________
conv1conv2d (Conv2D)         (None, 24, 44, 32)        288       
_________________________________________________________________
conv1conv2dbn (BatchNormaliz (None, 24, 44, 32)        96        
_________________________________________________________________
conv1conv2dact (Activation)  (None, 24, 44, 32)        0         
_________________________________________________________________
stem_br_11_maxpool_1 (MaxPoo (None, 22, 42, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 29568)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 59138     
Total para

### Train here

In [8]:
eyes_width = 90
eyes_height = 50
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [9]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

AllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "eye.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_sleepyCombination_eye.txt

(356,)

In [10]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [11]:
num_samples = 235578
batch_size = 200

In [12]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping



def LoadImageandLabel(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:

            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1

            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\eyes"


            #=====find all files in path=======    
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if "Bigframe" in file:
                        temp = os.path.join(r, file)
                        
                        Facefiles.append(temp)
                        
            
            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            for FileIndex in range(len(Facefiles)):
                #======================load label

                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 

                #print(TempLabelList[framenumber-1])
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=num_classes)
                LabelList.append(Labelarray)
                
                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Face_ImageDataList.append(ImgGray)
                 #====================================
               
                count = count +1
                if count>=batch_size:
                        #yield(np.array(Face_ImageDataList),np.array(LabelList))
                        yield ([np.array(Face_ImageDataList)],np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        LabelList = []

                        LabelList = []
best = 0
for i in range(80):
    EyeModel.fit_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile),
                    steps_per_epoch=ceil(num_samples / batch_size),
                    epochs=5,
                    #callbacks=[earlyStopping],
                    workers = 5,
                    shuffle=True
                    )
    result= EyeModel.evaluate_generator(LoadImageandLabelTesting(500,AllImageFolderTesting,AllLabelFileTesting), steps=ceil(167208 / 500),workers=5,verbose = 1)
    print(result[1])
    if(result[1]>best):
        print("save model")
        best = result[1]
        model_json = EyeModel.to_json()
        open('EyesModel_small.json', 'w').write(model_json)
        EyeModel.save_weights('EyesModel_small.h5')

        

Epoch 1/5
1178/1178 [==============================] - 40s 34ms/step - loss: 7.7705 - acc: 0.5150 7s - ETA: 2s - los - ETA: 1s - loss: 7
Epoch 2/5
1178/1178 [==============================] - 37s 32ms/step - loss: 7.8164 - acc: 0.5124
Epoch 3/5
1178/1178 [==============================] - 38s 32ms/step - loss: 7.7792 - acc: 0.5147
Epoch 4/5
1178/1178 [==============================] - 37s 32ms/step - loss: 7.8582 - acc: 0.5098 0s - loss: 7.8644 - acc: 0.50
Epoch 5/5
335/335 [==============================] - 182s 543ms/step
0.5166718797246956
save model
Epoch 1/5
1178/1178 [==============================] - 38s 32ms/step - loss: 7.7700 - acc: 0.5153 0s - loss: 7.7880 - acc: 0.51 - ETA: 0s - loss: 7.7921 - 
Epoch 2/5
1178/1178 [==============================] - 38s 33ms/step - loss: 7.8164 - acc: 0.5124
Epoch 3/5
1178/1178 [==============================] - 38s 33ms/step - loss: 7.7792 - acc: 0.5147 9s - lo - ETA: 7s - ETA:  - ETA: 3s - loss: 7.7 - ETA: 2s - loss: 7.7 - ETA: 1s - loss: 

KeyboardInterrupt: 

In [ ]:
 
model_json = model.to_json()
open('EyeModelNEW.json', 'w').write(model_json)
model.save_weights('EyeModelNEW.h5')


### Eyes2

### mouth

In [9]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Testing Image'

AllImageFolderTesting = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("mix" in directory):
            #print(directory)
            AllImageFolderTesting.append(os.path.join(r, directory))    
            
AllImageFolderTesting = AllImageFolderTesting[6:15]
for d in AllImageFolderTesting:
    print(d)
    
#llImageFolder = AllImageFolder[2:]
np.array(AllImageFolderTesting).shape

.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_sunglasses_mix


(9,)

In [10]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset'

AllLabelFileTesting = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "mouth.txt" in file :
            AllLabelFileTesting.append(os.path.join(r, file))

AllLabelFileTesting = AllLabelFileTesting[6:15]           
for f in AllLabelFileTesting:
    print(f)
    
#AllLabelFile = AllLabelFile [2:]
np.array(AllLabelFileTesting).shape

.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_nightglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_night_noglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_noglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_sunglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_glasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_nightglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_night_noglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_noglasses_mixing_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_sunglasses_mixing_mouth.txt


(9,)

In [11]:


eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 3
channel = 1

In [12]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping




def LoadImageandLabelTesting(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:
            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1

            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\face"
            

            #=====find all files in path=======    
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" in file:
                        temp = os.path.join(r, file)
                        Facefiles.append(temp)
                        
                        #Eyefiles.append(temp.replace("face", "eyes"))
                        #Mouthfiles.append(temp.replace("face", "mouth"))

            
            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            #Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            #Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            #Mouthfiles=Mouthfiles[:-2]
            #Eyefiles=Eyefiles[:-2]
            Facefiles=Facefiles[:-2]
            '''
            print(len(Facefiles))
            print(len(TempLabelList))
            '''
            for FileIndex in range(len(Facefiles)):
                #======================load label

                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 
                
                #print(framenumber)

                #if(judgeflag == 0 and TempLabelList[framenumber-1] == 1):
                #    judgeflag = 1
                    #print(framenumber-1)

                #Labelarray = np.array(LabelList)
                #print(len(TempLabelList))
                #print(framenumber-1)
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=3)
                LabelList.append(Labelarray)
                #print(LabelList)

                    #Labelarray = np.array(LabelList)
                    #Labelarray = to_categorical(Labelarray, num_classes=2)

                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                #ImgRGB = cv2.merge((ImgGray,ImgGray,ImgGray))# this line make gray image to RGB
                #TEMP_ImageDataList.append(ImgRGB)  
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                #print(ImgGray.shape)
                Face_ImageDataList.append(ImgGray)
                #print(np.array(Face_ImageDataList).shape)
                '''
                ImgGray = cv2.imread(Mouthfiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Mouth_ImageDataList.append(ImgGray)


                ImgGray = cv2.imread(Eyefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Eyes_ImageDataList.append(ImgGray)
                '''
                 #====================================
               
                count = count +1
                if count>=batch_size:

                        yield ([np.array(Face_ImageDataList)],np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        LabelList = []
                        
#      

Using TensorFlow backend.


In [13]:
%run ./Model.ipynb 

In [14]:
eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 3
channel = 1

In [30]:
mouth1 = Input(shape=(face_height,face_width,1))

x = conv2d(mouth1,32,3,2,'valid',True,name='conv1')
x = conv2d(x,32,3,1,'valid',True,name='conv2')
x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x_12 = conv2d(x,96,3,1,'valid',True,name='stem_br_12')

x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])

x = incresA(x,0.15,name = 'incresA_1')
x = incresA(x,0.15,name = 'incresA_2')
#x = incresA(x,0.15,name = 'incresA_3')

'''conv11 = Conv2D(32, kernel_size=(3, 3), activation='relu')(mouth1)
pool11 = MaxPooling2D(pool_size=(2, 2))(conv11)
conv12 = Conv2D(64, kernel_size=(3, 3), activation='relu')(pool11)
pool12 = MaxPooling2D(pool_size=(2, 2))(conv12)
conv13 = Conv2D(128, kernel_size=(3, 3), activation='relu',padding = 'same')(pool12)
pool13 = MaxPooling2D(pool_size=(2, 2))(conv13)
#flat1 = Flatten()(pool12)
#pool14 = GlobalAveragePooling2D(data_format='channels_last')(conv14)
flat1 = Flatten()(pool13)
output = Dense(num_classes, activation='softmax')(flat1)

'''
flat1 = Flatten()(x)

output = Dense(num_classes, activation='softmax')(flat1)




MouthModel = Model(inputs=[mouth1], outputs=output, name='inception_resnet_v2')
MouthModel.summary()
MouthModel.compile(loss='binary_crossentropy', optimizer=Adam(lr = 0.001), metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 31, 24, 32)   288         input_3[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 31, 24, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 31, 24, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [15]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

AllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "mouth.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_mouth.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_sleepyC

(356,)

In [16]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [17]:
num_samples = 235578
batch_size = 200

In [20]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping



def LoadImageandLabel(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:

            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1

            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\face"


            #=====find all files in path=======    
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" in file:
                        temp = os.path.join(r, file)
                        
                        Facefiles.append(temp)
                        

            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            for FileIndex in range(len(Facefiles)):
                #======================load label

                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 

                #print(TempLabelList[framenumber-1])
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=num_classes)
                LabelList.append(Labelarray)
                
                #================================
                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Face_ImageDataList.append(ImgGray)
                 #====================================
               
                count = count +1
                if count>=batch_size:
                        #yield(np.array(Face_ImageDataList),np.array(LabelList))
                        yield ([np.array(Face_ImageDataList)],np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        LabelList = []
best = 0
for i in range(12):
    model.fit_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile),
                    steps_per_epoch=ceil(num_samples / batch_size),
                    epochs=5,
                    #callbacks=[earlyStopping],
                    workers = 5,
                    shuffle=True
                    )
    result= model.evaluate_generator(LoadImageandLabelTesting(500,AllImageFolderTesting,AllLabelFileTesting), steps=ceil(167208 / 500),workers=5,verbose = 1)
    print(result[1])
    if(result[1]>best):
        print("save model")
        best = result[1]
        model_json = model.to_json()
        open('MouthModelNEW.json', 'w').write(model_json)
        model.save_weights('MouthModelNEW.h5')

        
    

Epoch 1/5
1178/1178 [==============================] - 273s 232ms/step - loss: 0.0456 - acc: 0.9833
Epoch 2/5
1178/1178 [==============================] - 260s 221ms/step - loss: 0.0382 - acc: 0.9860
Epoch 3/5
1178/1178 [==============================] - 257s 218ms/step - loss: 0.0410 - acc: 0.9849
Epoch 4/5
1178/1178 [==============================] - 269s 228ms/step - loss: 0.0390 - acc: 0.9858
Epoch 5/5
335/335 [==============================] - 46s 136ms/step
0.8821653568932328
save model
Epoch 1/5
1178/1178 [==============================] - 249s 211ms/step - loss: 0.0396 - acc: 0.9855
Epoch 2/5
1178/1178 [==============================] - 255s 217ms/step - loss: 0.0313 - acc: 0.9885
Epoch 3/5
1178/1178 [==============================] - 256s 218ms/step - loss: 0.0335 - acc: 0.9877
Epoch 4/5
1178/1178 [==============================] - 258s 219ms/step - loss: 0.0338 - acc: 0.9878
Epoch 5/5
335/335 [==============================] - 47s 140ms/step
0.8576364036135219
Epoch 1/5
1178/

KeyboardInterrupt: 

In [ ]:
model_json = model.to_json()
open('MouthModelNEW.json', 'w').write(model_json)
model.save_weights('MouthModelNEW.h5')

### Merge Model

In [1]:
%run ./Model.ipynb 

C:\Users\T-Rex\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [3]:
FaceInput = Input(shape = (1, 4, 2, 2048))
Facepool = Reshape((4, 2, 2048))(FaceInput)
Facepool = GlobalAveragePooling2D(data_format='channels_last')(Facepool)

flat0 = Flatten()(Facepool)

MouthInput = Input(shape = (1, 4, 2, 2048))
Mouthpool = Reshape((4, 2, 2048))(MouthInput)
Mouthpool = GlobalAveragePooling2D(data_format='channels_last')(Mouthpool)
flat1 = Flatten()(Mouthpool)

EyesInput = Input(shape = (1, 4, 2, 2048))
EyesPool = Reshape((4, 2, 2048))(EyesInput)
EyesPool = GlobalAveragePooling2D(data_format='channels_last')(EyesPool)
flat2 = Flatten()(EyesPool)

#=======================================================
FaceImageInput = Input(shape=(face_height,face_width,1))


#img_input = Input(shape=(face_height,face_width,1))
#img_input = Input(shape=(face_height,face_width,1))

x = conv2d(FaceImageInput,32,3,2,'valid',True,name='conv1')
x = conv2d(x,32,3,1,'valid',True,name='conv2')
x = conv2d(x,64,3,1,'valid',True,name='conv3')

x_11 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_11'+'_maxpool_1')(x)
x_12 = conv2d(x,96,3,1,'valid',True,name='stem_br_12')

x = Concatenate(axis=3, name = 'stem_concat_1')([x_11,x_12])

x_21 = conv2d(x,64,1,1,'same',True,name='stem_br_211')
x_21 = conv2d(x_21,64,[1,7],1,'same',True,name='stem_br_212')
x_21 = conv2d(x_21,64,[7,1],1,'same',True,name='stem_br_213')
x_21 = conv2d(x_21,96,3,1,'valid',True,name='stem_br_214')

x_22 = conv2d(x,64,1,1,'same',True,name='stem_br_221')
x_22 = conv2d(x_22,96,3,1,'valid',True,name='stem_br_222')

x = Concatenate(axis=3, name = 'stem_concat_2')([x_21,x_22])

x_31 = conv2d(x,192,3,1,'valid',True,name='stem_br_31')
x_32 = MaxPooling2D(3,strides=1,padding='valid',name='stem_br_32'+'_maxpool_2')(x)
x = Concatenate(axis=3, name = 'stem_concat_3')([x_31,x_32])
#Inception-ResNet-A modules
x = incresA(x,0.15,name='incresA_1')
x = incresA(x,0.15,name='incresA_2')
x = incresA(x,0.15,name='incresA_3')
#x = incresA(x,0.15,name='incresA_4')

#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(x)

x_red_12 = conv2d(x,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(x,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

x = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])

#Inception-ResNet-B modules
x = incresB(x,0.1,name='incresB_1')
x = incresB(x,0.1,name='incresB_2')
x = incresB(x,0.1,name='incresB_3')
x = incresB(x,0.1,name='incresB_4')
x = incresB(x,0.1,name='incresB_5')
#x = incresB(x,0.1,name='incresB_6')
#x = incresB(x,0.1,name='incresB_7')

#17 × 17 to 8 × 8 reduction module.
x_red_21 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_2')(x)

x_red_22 = conv2d(x,256,1,1,'same',True,name='x_red2_c11')
x_red_22 = conv2d(x_red_22,384,3,2,'valid',True,name='x_red2_c12')

x_red_23 = conv2d(x,256,1,1,'same',True,name='x_red2_c21')
x_red_23 = conv2d(x_red_23,256,3,2,'valid',True,name='x_red2_c22')

x_red_24 = conv2d(x,256,1,1,'same',True,name='x_red2_c31')
x_red_24 = conv2d(x_red_24,256,3,1,'same',True,name='x_red2_c32')
x_red_24 = conv2d(x_red_24,256,3,2,'valid',True,name='x_red2_c33')

x = Concatenate(axis=3, name='red_concat_2')([x_red_21,x_red_22,x_red_23,x_red_24])

#Inception-ResNet-C modules
x = incresC(x,0.2,name='incresC_1')
x = incresC(x,0.2,name='incresC_2')
#x = incresC(x,0.2,name='incresC_3')

#TOP
x = GlobalAveragePooling2D(data_format='channels_last')(x)
x = Dropout(0.6)(x)
#x = Dense(num_classes, activation='softmax')(x)
flat3 = Flatten()(x)

In [4]:
merge = Concatenate(axis=1, name='concat_final')([flat0, flat1, flat2, flat3])

# merge = Dense(2048, activation='relu')(merge)

merge = Dense(1024, activation='relu')(merge)

# merge = Dense(512, activation='relu')(merge)

merge = Dense(128, activation='relu')(merge)

# merge = Dense(64, activation='relu')(merge)

output = Dense(num_classes, activation='softmax')(merge)

model_Final = Model(inputs=[FaceInput, MouthInput, EyesInput, FaceImageInput], outputs=output, name='inception_resnet_v2')
model_Final.summary()
model_Final.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 64, 50, 1)    0                                            
__________________________________________________________________________________________________
conv1conv2d (Conv2D)            (None, 31, 24, 32)   288         input_4[0][0]                    
__________________________________________________________________________________________________
conv1conv2dbn (BatchNormalizati (None, 31, 24, 32)   96          conv1conv2d[0][0]                
__________________________________________________________________________________________________
conv1conv2dact (Activation)     (None, 31, 24, 32)   0           conv1conv2dbn[0][0]              
__________________________________________________________________________________________________
conv2conv2

In [5]:
'''
FaceInput = Input(shape=(1,2048))
FaceOut = Flatten()(FaceInput)

MouthInput = Input(shape=(1,21, 14, 384))
MouthReshape = Reshape((21, 14, 384))(MouthInput)
#35 × 35 to 17 × 17 reduction module.
x_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(MouthReshape)

x_red_12 = conv2d(MouthReshape,384,3,2,'valid',True,name='x_red1_c1')

x_red_13 = conv2d(MouthReshape,256,1,1,'same',True,name='x_red1_c2_1')
x_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')
x_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')

Mouth_Conv = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])


Mouth_Conv = Conv2D(32, kernel_size=(1, 1), activation='relu')(Mouth_Conv)
Mouth_Conv = Flatten()(Mouth_Conv)

Mouth_out = Dense(1024, activation='relu')(MouthInput)
#Mouth_out = Conv1D(32, kernel_size=(3), activation='relu')(MouthInput)

EyesInput = Input(shape=(1,2, 12, 384))
EyesReshape = Reshape((2, 12, 384))(EyesInput)

Eyes_out = Conv2D(32, kernel_size=(1, 1), activation='relu')(EyesReshape)

Eyes_out = Flatten()(Eyes_out)

merge = Concatenate(name='concat')([FaceOut, Mouth_Conv,Eyes_out])
merge = Flatten()(merge)

output = Dense(num_classes, activation='softmax')(merge)
'''

"\nFaceInput = Input(shape=(1,2048))\nFaceOut = Flatten()(FaceInput)\n\nMouthInput = Input(shape=(1,21, 14, 384))\nMouthReshape = Reshape((21, 14, 384))(MouthInput)\n#35 × 35 to 17 × 17 reduction module.\nx_red_11 = MaxPooling2D(3,strides=2,padding='valid',name='red_maxpool_1')(MouthReshape)\n\nx_red_12 = conv2d(MouthReshape,384,3,2,'valid',True,name='x_red1_c1')\n\nx_red_13 = conv2d(MouthReshape,256,1,1,'same',True,name='x_red1_c2_1')\nx_red_13 = conv2d(x_red_13,256,3,1,'same',True,name='x_red1_c2_2')\nx_red_13 = conv2d(x_red_13,384,3,2,'valid',True,name='x_red1_c2_3')\n\nMouth_Conv = Concatenate(axis=3, name='red_concat_1')([x_red_11,x_red_12,x_red_13])\n\n\nMouth_Conv = Conv2D(32, kernel_size=(1, 1), activation='relu')(Mouth_Conv)\nMouth_Conv = Flatten()(Mouth_Conv)\n\nMouth_out = Dense(1024, activation='relu')(MouthInput)\n#Mouth_out = Conv1D(32, kernel_size=(3), activation='relu')(MouthInput)\n\nEyesInput = Input(shape=(1,2, 12, 384))\nEyesReshape = Reshape((2, 12, 384))(EyesInput

In [6]:
'''
model_Final = Model(inputs=[FaceInput, MouthInput, EyesInput], outputs=output)
model_Final.summary()
model_Final.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
'''

"\nmodel_Final = Model(inputs=[FaceInput, MouthInput, EyesInput], outputs=output)\nmodel_Final.summary()\nmodel_Final.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])\n"

#### Train

##### load testing data

In [7]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Testing Image'

AllImageFolderTesting = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("mix" in directory):
            #print(directory)
            AllImageFolderTesting.append(os.path.join(r, directory))    
            
AllImageFolderTesting = AllImageFolderTesting[4:18]
for d in AllImageFolderTesting:
    print(d)
    
#llImageFolder = AllImageFolder[2:]
np.array(AllImageFolderTesting).shape

.\Dataset\Training_Evaluation_Dataset\Testing Image\004\004_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\022\022_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_nightnoglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_noglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\026\026_sunglasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\030\030_glasses_mix
.\Dataset\Training_Evaluation_Dataset\Testing Image\030\030_nightglasses_mix
.\Dataset\Training_Eva

(14,)

In [8]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset'

AllLabelFileTesting = []

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "drowsiness.txt" in file :
            AllLabelFileTesting.append(os.path.join(r, file))

AllLabelFileTesting = AllLabelFileTesting[4:18]           
for f in AllLabelFileTesting:
    print(f)
    
#AllLabelFile = AllLabelFile [2:]
np.array(AllLabelFileTesting).shape

.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\004\004_sunglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_glasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_nightglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_night_noglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_noglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\022\022_sunglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_glasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_nightglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_night_noglasses_mixing_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Evaluation Dataset\026\026_noglasses_mixing_drowsiness.txt
.\Dataset\Trai

(14,)

In [9]:
eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [10]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping




def LoadImageandLabel(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:
            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1
            
            Face_ImageInput = []
            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\face"
            

            #=====find all files in path=======  
            FaceImagefiles = []
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".npy" in file:
                        temp = os.path.join(r, file)
                        Facefiles.append(temp)
                        Eyefiles.append(temp.replace("face", "eyes"))
                        Mouthfiles.append(temp.replace("face", "mouth"))
            
            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            #Load image
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" in file:
                        temp = os.path.join(r, file)
                        FaceImagefiles.append(temp)
                        
            FaceImagefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            
            Facefiles = Facefiles[:-2]
            FaceImagefiles = FaceImagefiles[:-2]
            
            
            for FileIndex in range(len(Facefiles)):
                #======================load label
                
                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 
                
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=2)
                LabelList.append(Labelarray)
                #================================
                Eyes_ImageDataList.append(np.load(Eyefiles[FileIndex]))
                Mouth_ImageDataList.append(np.load(Mouthfiles[FileIndex]))
                Face_ImageDataList.append(np.load(Facefiles[FileIndex]))
                
                #load image
                ImgGray = cv2.imread(FaceImagefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Face_ImageInput.append(ImgGray)
                
                '''
                print(Facefiles[FileIndex])
                print(Mouthfiles[FileIndex])
                print(Eyefiles[FileIndex])
                print(FaceImagefiles[FileIndex])
                '''
                
                '''
                ImgGray = cv2.imread(Eyefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)

                
                Eyes_ImageDataList.append(EyeModel_Output)


                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                
                
                Face_ImageDataList.append(np.zeros((1,2048)))
                
                Mouth_ImageDataList.append(np.zeros((1,21, 14, 384)))
                '''
                '''
                print(MouthModel_Output.shape)
                print(EyeModel_Output.shape)
                print(FaceModel_Output.shape)
                print(np.array([[FaceModel_Output,MouthModel_Output,EyeModel_Output]]).shape)
                #print(type(MouthModel_Output))
                #Face_ImageDataList.append(ImgGray)
                '''
                 #====================================
               
                count = count +1
                
                if count>=batch_size:
                        batch = [np.array(Face_ImageDataList),np.array(Mouth_ImageDataList),np.array(Eyes_ImageDataList),np.array(Face_ImageInput)]
                        yield (batch,np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        Face_ImageInput = []
                        LabelList = []
            
                        

Using TensorFlow backend.


##### load training data

In [11]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

AllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "drowsiness.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Tra

(356,)

In [12]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [13]:
num_samples = 235578
batch_size = 200

eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [14]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping
import tensorflow as tf



def LoadImageandLabel(batch_size,AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    
    while True:
        LabelFileindex = 0
        for Originalpath in AllVideoImageFolder:
            labelPath = AllLabelFile[LabelFileindex]
            LabelFileindex = LabelFileindex + 1
            
            Face_ImageInput = []
            Face_ImageDataList = []
            Eyes_ImageDataList = []
            Mouth_ImageDataList = []
            #=====load all label in a video
            LabelList = []
            TempLabelList = []
            inputfile = open(labelPath, 'r').read()
            TempLabelList = list(map(int, inputfile))

            #=========================

            TEMP_ImageDataList = []

            path = Originalpath+"\\face"
            

            #=====find all files in path=======  
            FaceImagefiles = []
            Facefiles = []
            Eyefiles = []
            Mouthfiles = []
            
            # r=root, d=directories, f = files
            for r, d, f in os.walk(path):
                for file in f:
                    if ".npy" in file:
                        temp = os.path.join(r, file)
                        Facefiles.append(temp)
                        Eyefiles.append(temp.replace("face", "eyes"))
                        Mouthfiles.append(temp.replace("face", "mouth"))
            
            Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            #Load image
            for r, d, f in os.walk(path):
                for file in f:
                    if ".jpg" in file:
                        temp = os.path.join(r, file)
                        FaceImagefiles.append(temp)
                        
            FaceImagefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
            
            '''
            Facefiles = Facefiles[:-2]
            Eyefiles = Eyefiles[:-2]
            '''
            #print(Originalpath)
            for FileIndex in range(len(Facefiles)):
                #======================load label
                
                index = Facefiles[FileIndex].find('frame')
                framenumber =int(Facefiles[FileIndex][index+5:-4]) 
                
                Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=2)
                LabelList.append(Labelarray)
                #================================
                Eyes_ImageDataList.append(np.load(Eyefiles[FileIndex]))
                Mouth_ImageDataList.append(np.load(Mouthfiles[FileIndex]))
                Face_ImageDataList.append(np.load(Facefiles[FileIndex]))
                
                #load image
                ImgGray = cv2.imread(FaceImagefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                Face_ImageInput.append(ImgGray)
                
                '''
                print(Facefiles[FileIndex])
                print(Mouthfiles[FileIndex])
                print(Eyefiles[FileIndex])
                print(FaceImagefiles[FileIndex])
                print("----")
                '''
                '''
                ImgGray = cv2.imread(Eyefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)

                
                Eyes_ImageDataList.append(EyeModel_Output)


                ImgGray = cv2.imread(Facefiles[FileIndex],0)
                ImgGray = ImgGray.reshape(ImgGray.shape[0],ImgGray.shape[1],1)
                
                
                Face_ImageDataList.append(np.zeros((1,2048)))
                
                Mouth_ImageDataList.append(np.zeros((1,21, 14, 384)))
                '''
                '''
                print(MouthModel_Output.shape)
                print(EyeModel_Output.shape)
                print(FaceModel_Output.shape)
                print(np.array([[FaceModel_Output,MouthModel_Output,EyeModel_Output]]).shape)
                #print(type(MouthModel_Output))
                #Face_ImageDataList.append(ImgGray)
                '''
                 #====================================
               
                count = count +1
            
                if count>=batch_size:
                        batch = [np.array(Face_ImageDataList),np.array(Mouth_ImageDataList),np.array(Eyes_ImageDataList),np.array(Face_ImageInput)]
                        yield (batch,np.array(LabelList))
                        count=0
                        Face_ImageDataList = []
                        Eyes_ImageDataList = []
                        Mouth_ImageDataList = []
                        Face_ImageInput = []
                        LabelList = []
            
                        

                    
'''
model_Final.fit_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile),
                steps_per_epoch=ceil(num_samples / batch_size),
                epochs=100,
                #callbacks=[earlyStopping],
                workers = 5
                )
'''

best = 0
for i in range(80):
    model_Final.fit_generator(LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile),
                    steps_per_epoch=ceil(num_samples / batch_size),
                    epochs=1,
                    #callbacks=[earlyStopping],
                    workers = 5,
                    shuffle=True
                    )
    result= model_Final.evaluate_generator(LoadImageandLabelTesting(500,AllImageFolderTesting,AllLabelFileTesting), steps=ceil(167208 / 500),workers=5,verbose = 1)
    print(result[1])
    if(result[1]>best):
        print("save model")
        best = result[1]
        model_json = model_Final.to_json()
        open('FinalModelNEW.json', 'w').write(model_json)
        model.save_weights('FinalModelNEW.h5')


#LoadImageandLabel(batch_size,AllImageFolder,AllLabelFile)

Epoch 1/1
 380/1178 [========>.....................] - ETA: 22:30 - loss: 7.6240 - acc: 0.5225

KeyboardInterrupt: 

### SVM

In [1]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

EyeAllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "eye.txt" in file :
            EyeAllLabelFile.append(os.path.join(r, file))
        
for f in EyeAllLabelFile:
    print(f)
np.array(EyeAllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_eye.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_sleepyCombination_eye.txt

(356,)

In [2]:
# load label

import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset'

AllLabelFile = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if "drowsiness.txt" in file :
            AllLabelFile.append(os.path.join(r, file))
        
for f in AllLabelFile:
    print(f)
np.array(AllLabelFile).shape

.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_sleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_slowBlinkWithNodding_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\glasses\001_yawning_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_sleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_slowBlinkWithNodding_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\nightglasses\001_yawning_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Training Dataset\001\night_noglasses\001_nonsleepyCombination_drowsiness.txt
.\Dataset\Training_Evaluation_Dataset\Tra

(356,)

In [3]:
import os
import numpy as np

#path = '.\Dataset\Training_Evaluation_Dataset\Training Dataset\\'
path = '.\Dataset\Training_Evaluation_Dataset\Training Image'

AllImageFolder = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for directory in d:
        if ("nonsleepyCombination" in directory or "sleepyCombination" in directory or "slowBlinkWithNodding"  in directory or "yawning" in directory)   :
            #print(directory)
            AllImageFolder.append(os.path.join(r, directory))
        
for d in AllImageFolder:
    print(d)
np.array(AllImageFolder).shape

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

(356,)

In [4]:
num_samples = 235578
batch_size = 200

eyes_width = 45
eyes_height = 25
mouth_width = 35
mouth_height = 25
face_width = 50
face_height = 64
num_classes = 2
channel = 1

In [16]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping
import tensorflow as tf



def LoadImageandLabel(AllVideoImageFolder,AllLabelFile):
    #print(Originalpath)
    count=0
    

    LabelFileindex = 0
    

    Face_ImageDataList = []
    Eyes_ImageDataList = []
    Mouth_ImageDataList = []
    
    LabelList = []
    for Originalpath in AllVideoImageFolder:
        labelPath = AllLabelFile[LabelFileindex]
        
        eyelabelPath = EyeAllLabelFile[LabelFileindex]
        facelabelPath = EyeAllLabelFile[LabelFileindex].replace("face", "head")
        mouthlabelPath = EyeAllLabelFile[LabelFileindex].replace("face", "mouth")
        
        LabelFileindex = LabelFileindex + 1
        #=====load all label in a video
        TempLabelList = []
        EyeTempLabelList = []
        FaceTempLabelList = []
        MouthTempLabelList = []
        
        inputfile = open(labelPath, 'r').read()
        TempLabelList = list(map(int, inputfile))
        
        
        Eyeinputfile = open(eyelabelPath, 'r').read()
        EyeTempLabelList = list(map(int, Eyeinputfile))
        
        Faceinputfile = open(facelabelPath, 'r').read()
        FaceTempLabelList = list(map(int, Faceinputfile))
        
        Mouthinputfile = open(mouthlabelPath, 'r').read()
        MouthTempLabelList = list(map(int, Mouthinputfile))
        
        #=========================


        path = Originalpath+"\\face"

        
        #=====find all files in path=======  
        Facefiles = []
        Eyefiles = []
        Mouthfiles = []

        # r=root, d=directories, f = files
        
        for r, d, f in os.walk(path):
            for file in f:
                if "_Prob.npy" in file:
                    temp = os.path.join(r, file)
                    Facefiles.append(temp)
                    Eyefiles.append(temp.replace("face", "eyes"))
                    Mouthfiles.append(temp.replace("face", "mouth"))

        Facefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        Eyefiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        Mouthfiles.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
        
        '''
        Facefiles = Facefiles[:-2]
        Eyefiles = Eyefiles[:-2]
        '''
        print(Originalpath)
        for FileIndex in range(len(Facefiles)):
            
            #======================load label

            index = Facefiles[FileIndex].find('frame')
            
            framenumber =int(Facefiles[FileIndex][index+5:-9]) 

            Labelarray = to_categorical(TempLabelList[framenumber-1], num_classes=2)
            LabelList.append(Labelarray)
            
            EyeLabelarray = to_categorical(EyeTempLabelList[framenumber-1], num_classes=2)
            Eyes_ImageDataList.append(EyeLabelarray)
            
            MouthLabelarray = to_categorical(EyeTempLabelList[framenumber-1], num_classes=3)
            Mouth_ImageDataList.append(MouthLabelarray)
            
            HeadLabelarray = to_categorical(FaceTempLabelList[framenumber-1], num_classes=3)
            Face_ImageDataList.append(HeadLabelarray)
            
            #================================
            #Eyes_ImageDataList.append(np.load(Eyefiles[FileIndex]))
            #Mouth_ImageDataList.append(np.load(Mouthfiles[FileIndex]))
            #Face_ImageDataList.append(np.load(Facefiles[FileIndex]))
            
            
            
            '''
            print(Facefiles[FileIndex])
            print(Mouthfiles[FileIndex])
            print(Eyefiles[FileIndex])
            print("----")
            '''
    return Face_ImageDataList ,Eyes_ImageDataList ,Mouth_ImageDataList ,LabelList
            
Facefiles = []
Eyefiles = [] 
Mouthfiles = [] 
LabelList = []    
Facefiles ,Eyefiles ,Mouthfiles ,LabelList = LoadImageandLabel(AllImageFolder,AllLabelFile)

.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\glasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\001\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\001\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Trainin

.\Dataset\Training_Evaluation_Dataset\Training Image\009\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\009\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\009\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\009\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\009\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\009\night_noglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\009\noglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\009\sunglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training

.\Dataset\Training_Evaluation_Dataset\Training Image\023\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\023\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\023\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\023\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\023\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\023\night_noglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\023\noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\023\noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\023\noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\023\noglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\023\sunglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training

.\Dataset\Training_Evaluation_Dataset\Training Image\034\nightglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\034\nightglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\034\nightglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\034\night_noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\034\night_noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\034\night_noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\034\night_noglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training Image\034\noglasses\nonsleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\034\noglasses\sleepyCombination
.\Dataset\Training_Evaluation_Dataset\Training Image\034\noglasses\slowBlinkWithNodding
.\Dataset\Training_Evaluation_Dataset\Training Image\034\noglasses\yawning
.\Dataset\Training_Evaluation_Dataset\Training 

In [17]:
print(np.array(Facefiles).shape)
print(np.array(Eyefiles).shape)
print(np.array(Mouthfiles).shape)
print(np.array(LabelList).shape)

(235578, 3)
(235578, 2)
(235578, 3)
(235578, 2)


In [7]:
import numpy as np
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import cv2
from math import ceil

from keras.callbacks import EarlyStopping
import tensorflow as tf
LabelList = np.load(".\\Dataset\\Training_Evaluation_Dataset\\Prob\\Label.npy")
Mouthfiles = np.load(".\\Dataset\\Training_Evaluation_Dataset\\Prob\\Mouth.npy")
#Eyefiles = np.load(".\\Dataset\\Training_Evaluation_Dataset\\Prob\\Eyes.npy")
Facefiles = np.load(".\\Dataset\\Training_Evaluation_Dataset\\Prob\\Face.npy")
print(np.array(Facefiles).shape)
print(np.array(Eyefiles).shape)
print(np.array(Mouthfiles).shape)
print(np.array(LabelList).shape)


(235578, 3)
(235578, 2)
(235578, 3)
(235578, 2)


In [18]:
print(Facefiles[0])
print(Eyefiles[0])
print(Mouthfiles[0])

[1. 0. 0.]
[1. 0.]
[1. 0. 0.]


In [19]:

FeatureList = np.concatenate((Mouthfiles, Facefiles), axis = 1)
FeatureList = np.concatenate((FeatureList, Eyefiles), axis = 1)

LabelList = np.array(LabelList)

In [20]:
FeatureList[0]

array([1., 0., 0., 1., 0., 0., 1., 0.], dtype=float32)

In [21]:
print(FeatureList.shape)
print(LabelList.shape)

(235578, 8)
(235578, 2)


In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras.layers import Input,Dense,LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers, activations
from tensorflow.keras.layers import Reshape


feature_input = Input(shape=(8,))

#out1 = LSTM(128)(feature_input)
output = Dense(2, activation='softmax')(feature_input)

model = Model(inputs=feature_input, outputs=output, name='model_classification')
model.summary()
model.compile(loss='binary_crossentropy', optimizer=Adam(lr = 0.02), metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 8)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 18        
Total params: 18
Trainable params: 18
Non-trainable params: 0
_________________________________________________________________


In [23]:
model.fit(FeatureList, LabelList, batch_size=20, epochs=10,verbose =1)


Epoch 1/10
235578/235578 [==============================] - 17s 71us/step - loss: 0.2002 - acc: 0.9481
Epoch 2/10
235578/235578 [==============================] - 17s 71us/step - loss: 0.1996 - acc: 0.9486
Epoch 3/10
235578/235578 [==============================] - 17s 71us/step - loss: 0.1998 - acc: 0.9486
Epoch 4/10
 41100/235578 [====>.........................] - ETA: 13s - loss: 0.1990 - acc: 0.9489

KeyboardInterrupt: 

In [23]:

model_json = model.to_json()
open('Support.json', 'w').write(model_json)
model.save_weights('Support.h5')

### SVM

In [ ]:

class_labels = np.argmax(LabelList, axis=1) 

In [ ]:
from sklearn import svm
clf = svm.SVC(gamma='auto',verbose = 2)
clf.fit(FeatureList, class_labels)

[LibSVM]

In [ ]:
import pickle #pickle模块

with open('Prob\\clf.pickle', 'wb') as f:
    pickle.dump(clf, f)


with open('Prob\\lclf.pickle', 'rb') as f:
    clf2 = pickle.load(f)
    print(clf2.predict(X[0:1]))

# [0]